### Imports

In [ ]:
import pyspark.sql.functions as F
from utils.params import SCHEMA, URL_PARAMETROS, URL_PERIODOS, INSTITUICOES_ESPECIFICAS, PRODUTOS_BKP
from utils.helper import *
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np

### SparkSession

In [ ]:
from pyspark.sql import SparkSession


spark = (
    SparkSession.builder
    .appName("default")
    .getOrCreate()
)

#https://www.bcb.gov.br/api/servico/sitebcb/historicotaxajurosdiario/TodosCampos?filtro=(codigoSegmento%20eq%20%272%27)%20and%20(codigoModalidade%20eq%20%27211101%27)%20and%20(InicioPeriodo%20eq%20%272025-03-24%27)

### Parâmetros

In [ ]:
PRODUTOS = obter_produtos_bacen(URL_PARAMETROS)
PRODUTOS = PRODUTOS if PRODUTOS else PRODUTOS_BKP

df_produtos = (
    spark.createDataFrame(PRODUTOS)
    .selectExpr(
        "codigoModalidade as CD_MODL",
        "Modalidade as DS_MODL",
        "codigoSegmento as CD_SEGM",
        "Segmento as NM_SEGM",
        "tipoModalidade as TP_MODL",
    )
    .withColumn("DS_TIPO_PERI_MODL",
                F.when(
                    F.col("TP_MODL") == "M", F.lit("Mensal")
                )
                .when(
                    F.col("TP_MODL") == "D", F.lit("Diária")
                )
                .when(
                    F.col("TP_MODL") == "A", F.lit("Anual")
                )
                .otherwise(
                    F.lit(None)
                )
    )
    .withColumn("dat_ref_carga", F.lit(date.today()))
    .withColumn("dh_exec", F.lit(datetime.now()))
)
df_produtos.show()

### Períodos

In [ ]:
PERIODOS, PERIODOS_INGEST = obter_periodos_bacen(URL_PERIODOS)

print(f"{PERIODOS['inicio']}")
print(f"{PERIODOS['final']}")

df_periodos = (
    spark.createDataFrame(PERIODOS_INGEST)
    .selectExpr(
        "Periodo as DS_PERI",
        "InicioPeriodo as DT_INIC_PERI",
        "FimPeriodo as DT_FINA_PERI",
        "tipoModalidade as TP_MODL",
    )
    .withColumn("dat_ref_carga", F.lit(date.today()))
    .withColumn("dh_exec", F.lit(datetime.now()))
)

df_periodos.show()

### Data única

In [ ]:

df_list = extrair_dados_bacen_data_unica(PRODUTOS, PERIODOS, data_alvo="2025-03-26")

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

In [ ]:
df.select("QT_DIA_APROX").distinct().show(truncate=False)

In [ ]:
import pyspark.sql.functions as F

df.filter(F.col("NR_POSI") == 1).show(truncate=False)

### Mês específico

In [ ]:

df_list = extrair_dados_bacen_mes(PRODUTOS, PERIODOS, mes="2025-01")

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

In [ ]:
df.count()

In [ ]:
df.select("dat_ref_carga").distinct().show(truncate=False)

### Ano específico

In [ ]:
df_list = extrair_dados_bacen_ano(PRODUTOS, PERIODOS, ano="2025")

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

### Ano específico com range de meses

In [ ]:
df_list = extrair_dados_bacen_ano(PRODUTOS, ano="2024", mes_final = 3)

df = spark.createDataFrame(df_list, schema=SCHEMA)

In [ ]:
df.show(truncate=False)

## Gráficos

In [ ]:
pddf = (
    df.toPandas()
)

pddf = (
    pddf[
        [
            "NM_BANK",
            "NM_SEGM",
            "DS_MODL",
            "NR_POSI",
            "IN_INIC_PERI_EXAT",
            "DT_APROX",
            "QT_DIA_APROX",
            "VL_TAXA_JURO_AM",
            "VL_TAXA_JURO_AA",
            "dat_ref_carga"
        ]
    ]
    .rename(
        columns={"dat_ref_carga":"DT_TAXA"}
    )
)
cols = pddf.columns
print(pddf.columns)
pddf.head(20)

#### Taxas de juros (AM) por instituição, ao longo do tempo, em cada modalidade

In [ ]:
taxas_df = (
    pddf[
        ["NM_BANK","DT_TAXA", "DS_MODL", "VL_TAXA_JURO_AM", "NR_POSI"]
    ].copy()
)
taxas_df["DT_TAXA"] = pd.to_datetime(taxas_df["DT_TAXA"], format='%Y-%m-%d')
cols = taxas_df.columns
modalidades = taxas_df["DS_MODL"].copy().drop_duplicates().to_list()


print(f"Instituições distintas: {taxas_df['NM_BANK'].copy().drop_duplicates().count()}")
print(f"Datas distintas: {taxas_df['DT_TAXA'].copy().drop_duplicates().count()}")
print(f"Modalidades distintas: {len(modalidades)}")

print(f"\nSchema:\n{taxas_df.dtypes}")


taxas_df.head()

In [ ]:
for modl in modalidades:
    modl_df = taxas_df[taxas_df["DS_MODL"] == modl]
    qtd_instituicoes = modl_df["NM_BANK"].drop_duplicates().count()
    print(f"{modl}: {qtd_instituicoes} instituições")

    lista_top_bancos_por_modl = (
        modl_df
        .groupby(["NM_BANK"])
        .agg({
            "NR_POSI": ["mean","count"]
        })
        .reset_index()
    )

    lista_top_bancos_por_modl.columns = ['NM_BANK', 'NR_POSI_mean', 'NR_POSI_count']

    lista_top_bancos_por_modl = (
        lista_top_bancos_por_modl[lista_top_bancos_por_modl["NR_POSI_count"] >= 15]
        .sort_values(by="NR_POSI_mean", ascending=True)
        ["NM_BANK"]
        .tolist()
    )

    lista_bancos_especificos_modl = modl_df[modl_df["NM_BANK"].isin(INSTITUICOES_ESPECIFICAS)].sort_values(by="VL_TAXA_JURO_AM", ascending=True)["NM_BANK"].drop_duplicates().tolist()

    plt.figure(figsize=(20, 6))

    for banco in lista_bancos_especificos_modl[:10]:
        banco_df = modl_df[modl_df["NM_BANK"] == banco]
        plt.plot(banco_df["DT_TAXA"], banco_df["VL_TAXA_JURO_AM"], label=banco, marker='o', alpha=0.8)

    # Config gráfico
    plt.title(f"{modl}: ao mês")
    plt.ylabel("Taxa")
    plt.xlabel("Data")
    plt.legend(title="Instituições")
    plt.xticks(rotation=45)
    plt.grid(True)

    plt.show()
